In [1]:
import numpy as np
import pandas as pd
import torch
import torchvision
from torchvision.transforms import transforms
import torch.nn as nn
import os
import glob
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
device= torch.device('cuda'if torch.cuda.is_available else 'cpu')

In [3]:
print(device)

cuda


In [4]:
Transform= transforms.Compose([transforms.Resize((150,150)),transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize([0.5,0.5,0.5]
                                                  ,[0.5,0.5,0.5])
                              ])

In [5]:
train="C:\\Users\\navee\\OneDrive\\Desktop\\concrete\\train"
test="C:\\Users\\navee\\OneDrive\\Desktop\\concrete\\test"

In [6]:
train_loader= DataLoader(torchvision.datasets.ImageFolder(train, transform=Transform), batch_size=100, shuffle=True)
test_loader= DataLoader(torchvision.datasets.ImageFolder(test, transform=Transform), batch_size=100, shuffle=True)

In [7]:
#neuralNetwork


class NeuralNetwork(nn.Module):
    
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=2)
        
        
        
        #Feed forwad function
        
    def forward(self,input):

        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output
        
        
        
        

In [8]:
model= NeuralNetwork()

In [9]:
lossy=nn.CrossEntropyLoss()

In [10]:
optimizer=Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

In [11]:
import glob

In [12]:
trainlen= len(glob.glob(train+'*/**/*.jpg'))
testlen= len(glob.glob(test+'*/**/*.jpg'))

In [13]:
num_epoch=2

In [14]:
best_accuracy=0.0

for epoch in range(num_epoch):
    
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    
    for i, (images, labels) in enumerate(train_loader):
        images=images
        labels=labels
    
        
        optimizer.zero_grad()
        
        output=model(images)
        loss=lossy(output, labels)
        loss.backward()
        optimizer.step()
        
        _,prediction=torch.max(output, 1)
        
        train_accuracy+=int(torch.sum(prediction==labels))
        
    train_accuracy=train_accuracy/trainlen        
    
    model.eval()
    test_accuracy=0.0
        
    for i, (images, labels) in enumerate(test_loader):
        images=images
        labels=labels
        
        
        outputs=model(images)
        _,prediction=torch.max(outputs, 1)
        
        test_accuracy+=int(torch.sum(prediction==labels))   
        
        
    test_accuracy=test_accuracy/testlen



    print('Epoch: '+str(epoch)+'Train Accuracy: '+str(train_accuracy)+'Test Accuracy: '+ str(test_accuracy))
    
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy
    
    
        
        

C:\Users\navee\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


KeyboardInterrupt: 